In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as integrate
from scipy.stats import uniform, norm
import scipy.stats as sps
from random import choices


In [ ]:
resultats = GibbsABC(X, N, M, P, P2, Npts, objectifeps, temps_max, T)

IndexError: tuple index out of range

In [15]:
def changepoids(simui, nouveps, eps, l, M):
    if simui[0]==0:
        return(simui)
    if sum(simui[l+3:l+M+3]<= eps) == 0:
        simui[1] = 0
        return(simui)
    simui[1]=simui[0]*sum(simui[l+3:l+M+3]<= nouveps)/sum(simui[l+3:l+M+3]<= eps)
    return(simui)
    
def changementeps(simu, eps, alpha, n, M, l):
    epsilons = np.sort(simu[:,l+3:M+l+3][simu[:,l+3:M+l+3]<eps])[::-1]
    #print(epsilons)
    nbops = len(epsilons)
    print("nbops")
    print(nbops)
    ESS = 1/sum(simu[:,0]**2)
    k = 0
    while (k<nbops-1):
        nouveps = epsilons[k]
        a=True
        simu = np.apply_along_axis(lambda x : changepoids(x, nouveps, eps,l, M),1,simu)
        simu[:,1] = simu[:,1] / sum(simu[:,1])
        nouvESS = 1/sum(simu[:,1]**2)
        if nouvESS < (alpha * ESS) :
            simu[:,0] = simu[:,1]
            simu[:,1] = np.zeros(n)
            return(simu, nouveps, nouvESS)
        k += 1
    return(simu,eps,ESS)

def chgtpar(simui, sd, eps, dprior, dist, n, M, l):
    var = np.random.multivariate_normal(np.zeros(l),cov=sd)
    simu = simui + np.concatenate((np.zeros(2),[np.mean(var)],var,np.zeros(M)))
    #print(simu)
    simu[l+3:l+M+3] = dist(simu)
    snew = sum(simu[l+3:l+M+3] <= eps)
    sold = sum(simui[l+3:l+M+3] <= eps)
    u = np.random.uniform(1)
    #print(distances, distancesi)
    if sold == 0:
        return(simui)
    if (u < (snew / sold * dprior(simu) / dprior(simui))):
        return(simu)
    else :
        return(simui)
        

def smc(data, n, M, T, objectifeps, alpha, rprior, dprior, dist, l):
    simu = np.concatenate((np.zeros((n,2)),rprior(n)), axis = 1)
    distances = np.apply_along_axis(dist,1,simu)
    simu = np.concatenate((simu,distances), axis = 1)
    j=1
    simu[:,0] = np.array([1/n]*n)
    ESS = n
    eps = np.max(simu[:,l+3:M+l+3])
    histeps = [eps]
    while ((j<T) and (eps>objectifeps)):
        (simu, eps, ESS) = changementeps(simu, eps, alpha, n, M, l)
        histeps.append(eps)
        if (ESS < n / 2) :
            simu = np.array(choices(simu, weights = simu[:,0], k = n))
            simu[:,0] = np.array([1/n]*n)
            ESS = n
            simu[:,l+3:M+l+3] = np.apply_along_axis(dist,1,simu)
        print(distances.shape)
        sd = 2 * np.cov(np.transpose(simu[:,3:l+3]))
        simu = np.apply_along_axis(lambda x : chgtpar(x, sd, eps, dprior, dist, n, M, l),1,simu)
        print(j,eps)
        j += 1
    simu = choices(simu, weights = simu[:,0], k = n)
    #poids = np.array([1/n]*n)
    ESS = n
    distances = np.array([dist(i) for i in simu]*M)
    return(simu, histeps)

In [16]:
l = 20
M = 20
n = 30
ndata = 30
genvar = np.random.normal
data = genvar(-5, 1, (ndata,l))

def rprior(n):
    means = np.random.uniform(-10,10,size = (n,1))
    res = np.apply_along_axis(lambda x : genvar(x,1,l),1,means)
    return(np.concatenate((means,res),axis=1))

statdata = np.apply_along_axis(lambda x : np.percentile(x, np.arange(10, 91, 10)),0,data)

def dprior(simui):
    m = simui[2]
    return(np.sum(np.apply_along_axis(lambda x : norm.pdf(x, loc=m, scale=1),0,np.expand_dims(simui[3:l+3], axis = 1))))

def dist(simui):
    lev = np.arange(0.1, 0.91, 0.1)
    somme = np.apply_along_axis(lambda i : np.sum(np.abs(np.apply_along_axis(lambda x : np.percentile(genvar(x, 1, ndata), np.arange(10,91,10)),0,np.expand_dims(simui[3:l+3], axis = 0))-statdata)),1,np.zeros((M,1)))
    return(somme)


In [ ]:
l = 20
M = 20
n = 30
ndata = 30
genvar = np.random.normal

def rprior(n, lower = -4, upper = 4):
    alpha = np.random.uniform(lower, upper, size=(n,1))
    thet = np.apply_along_axis(lambda x : genvar(x,1,l),1,alpha)
    return(np.concatenate((alpha,thet),axis=1))

def dprior(simui):
    m = simui[2]
    if (abs(m) < 4):
        return(1 / 4 * prod(dnorm(x[seq_len(l) + 1], x[1])))
    else:
        return(0)

def model(pars, data, n = 20, K = 10):
    temp <- matrix(rnorm(n * K, rep(pars[seq_len(n) + 1], K), 1), nrow = n)
    return(sum(abs(rowMeans(temp - data))))

In [17]:
smc(data, n, M, 50, 150, 0.9, rprior, dprior, dist, l)

nbops
599
(30, 20)
1 1962.2338729675093
nbops
539
(30, 20)
2 1492.3321890684065
nbops
500
(30, 20)
3 1327.6861241958618
nbops
460
(30, 20)
4 813.051312101256
nbops
410
(30, 20)
5 768.3194806283318
nbops
367
(30, 20)
6 676.0379967247147
nbops
339
(30, 20)
7 633.1751045703654
nbops
582
(30, 20)
8 506.8043426580157
nbops
543
(30, 20)
9 457.3917917437409
nbops
502
(30, 20)
10 429.0374026825426
nbops
461
(30, 20)
11 385.0162627837892
nbops
438
(30, 20)
12 334.197184926752
nbops
407
(30, 20)
13 299.82830922354
nbops
327
(30, 20)
14 295.011779295491
nbops
577
(30, 20)
15 269.2761058839698
nbops
548
(30, 20)
16 245.99698731623897
nbops
531
(30, 20)
17 218.08985904881712
nbops
413
(30, 20)
18 214.0290729769902
nbops
434
(30, 20)
19 209.06898586850738
nbops
409
(30, 20)
20 200.80365089887835
nbops
382
(30, 20)
21 180.24629169735306
nbops
593
(30, 20)
22 163.02798495148403
nbops
478
(30, 20)
23 159.12327915002948
nbops
415
(30, 20)
24 156.15977701498073
nbops
401
(30, 20)
25 153.77197376423575
nb

([array([ 5.46224004e-02,  0.00000000e+00, -4.63697998e+00, -4.34212753e+00,
         -4.23793455e+00, -7.17778866e+00, -5.38381915e+00, -4.69112058e+00,
         -3.83502946e+00, -5.72912606e+00, -5.38469099e+00, -3.70635814e+00,
         -4.76721989e+00, -4.92814541e+00, -3.50772826e+00, -4.21754335e+00,
         -5.60252766e+00, -4.88827000e+00, -4.75275109e+00, -4.55276148e+00,
         -5.27418724e+00, -4.79849541e+00, -5.26297225e+00,  1.18727019e+02,
          1.24250652e+02,  1.19114977e+02,  1.26449459e+02,  1.21307590e+02,
          1.23090810e+02,  1.27767173e+02,  1.15063719e+02,  1.22957792e+02,
          1.28443457e+02,  1.27191457e+02,  1.11874857e+02,  1.13731099e+02,
          1.25423471e+02,  1.31893616e+02,  1.23276044e+02,  1.29018687e+02,
          1.18975085e+02,  1.05578794e+02,  1.12234278e+02]),
  array([ 6.19053871e-02,  0.00000000e+00, -4.70430524e+00, -4.16131444e+00,
         -4.39302855e+00, -4.71425531e+00, -5.41169334e+00, -4.12185979e+00,
         -3.52

In [118]:
np.shape(rprior(50))

(50, 40)

In [102]:
simui = np.concatenate((np.zeros(3),genvar(-5.5, 1, l)))
np.apply_along_axis(np.sum(np.abs(np.apply_along_axis(lambda x : np.percentile(genvar(x, 1, ndata), np.arange(10,91,10)),0,np.expand_dims(simui[3:l+3], axis = 0))-statdata)),1,np.zeros((1,M)))

TypeError: 'numpy.float64' object is not callable

In [169]:
np.random.normal(0, 1, (20,l))

array([[ 1.14300981e+00,  7.23000039e-01, -9.93657375e-02,
        -1.18467928e+00,  1.16173018e+00, -2.29294434e+00,
         7.41899761e-01,  1.85956366e-01, -1.09490732e+00,
        -4.10943157e-01,  9.50964283e-02,  5.95530641e-01,
         6.41885243e-01, -8.47788483e-01, -4.47915231e-01,
        -1.35739315e+00, -7.37665918e-01, -4.34445147e-01,
        -3.91672255e-01,  5.73217372e-01, -2.88070483e-01,
         3.55776367e-01, -2.99554238e-01,  1.45798614e+00,
         1.63922154e+00, -6.78473799e-01,  1.09049887e+00,
        -1.00425997e+00, -2.12571256e-01,  8.57885088e-01,
        -7.87559646e-01, -3.58762907e-01,  1.08809418e+00,
        -2.38742935e+00,  4.28606226e-03, -1.08117323e+00,
         5.43706218e-01, -1.12549646e-01,  2.46561950e+00,
         1.09513812e+00, -3.22118983e-01,  7.86767243e-01,
        -1.30025173e+00, -1.17708705e+00, -1.47932316e+00,
        -7.09729489e-01,  1.52335685e+00, -7.96703998e-01,
        -1.18088898e+00, -1.14511035e+00],
       [ 1.14

In [96]:
lev = np.arange(0.1, 0.91, 0.1)
np.array([[sps.norm(0,1).ppf(lev[j])for j in range(9)] for i in range(20)])

array([[-1.28155157, -0.84162123, -0.52440051, -0.2533471 ,  0.        ,
         0.2533471 ,  0.52440051,  0.84162123,  1.28155157],
       [-1.28155157, -0.84162123, -0.52440051, -0.2533471 ,  0.        ,
         0.2533471 ,  0.52440051,  0.84162123,  1.28155157],
       [-1.28155157, -0.84162123, -0.52440051, -0.2533471 ,  0.        ,
         0.2533471 ,  0.52440051,  0.84162123,  1.28155157],
       [-1.28155157, -0.84162123, -0.52440051, -0.2533471 ,  0.        ,
         0.2533471 ,  0.52440051,  0.84162123,  1.28155157],
       [-1.28155157, -0.84162123, -0.52440051, -0.2533471 ,  0.        ,
         0.2533471 ,  0.52440051,  0.84162123,  1.28155157],
       [-1.28155157, -0.84162123, -0.52440051, -0.2533471 ,  0.        ,
         0.2533471 ,  0.52440051,  0.84162123,  1.28155157],
       [-1.28155157, -0.84162123, -0.52440051, -0.2533471 ,  0.        ,
         0.2533471 ,  0.52440051,  0.84162123,  1.28155157],
       [-1.28155157, -0.84162123, -0.52440051, -0.2533471 ,  0

In [95]:
data = np.random.normal(0, 1, (20,l))

def rprior(n):
    means = np.random.uniform(-10,10,size = n)
    res = [0]*n
    for i in range(n):
        res[i] = np.random.normal(means[i],1,l)
    return(res,means)

np.array([np.percentile(data[i], np.arange(10, 91, 10)) for i in range(20)])

array([[-1.22632763, -0.96757533, -0.39301757, -0.20397749,  0.14693889,
         0.37491585,  0.6289771 ,  0.92455755,  1.38612635],
       [-1.44296325, -0.92307548, -0.56156128, -0.39494026, -0.2664423 ,
        -0.00636477,  0.30104914,  0.79924288,  1.24402695],
       [-1.04964449, -0.70018076, -0.39034465, -0.2240184 , -0.01453115,
         0.14606471,  0.33530056,  0.62722259,  1.30862775],
       [-1.49909209, -1.0028768 , -0.79626747, -0.31251036,  0.03944439,
         0.21273599,  0.39171022,  0.62138397,  1.10633739],
       [-1.00312275, -0.5279426 , -0.15478898, -0.02036877,  0.10632089,
         0.27521066,  0.5321973 ,  0.86913597,  1.00471078],
       [-1.02707699, -0.59789934, -0.33801833,  0.11657606,  0.30406687,
         0.40774515,  0.58181003,  0.84903717,  1.39213776],
       [-1.44728498, -1.1295602 , -0.70792207, -0.47793317, -0.11071702,
         0.15321368,  0.32667567,  0.83899428,  1.06471114],
       [-0.97542639, -0.34560624, -0.14772612,  0.1749131 ,  0

In [119]:
l = 50
data = np.random.normal(0, 1, (20,l))

def rprior(n):
    means = np.random.uniform(-10,10,size = n)
    res = [0]*n
    for i in range(n):
        res[i] = np.random.normal(means[i],1,l)
    return(res,means)

statdata = np.array([np.percentile(data[i], np.arange(10, 91, 10)) for i in range(20)])

def dprior(simu):
    return(np.sum(norm.pdf(simu, loc=simu[1], scale=1)))

def dist(simu):
    lev = np.arange(0.1, 0.91, 0.1)
    quantiles = np.array([[sps.norm(simu[i],1).ppf(lev[j])for j in range(9)] for i in range(20)])
    return(np.sum(abs(quantiles - statdata)))

dist(np.random.normal(0, 1, (l)))

156.34695097571228

In [7]:
np.tile([1,2,3],(3,1))[:,2]


array([3, 3, 3])